In [ ]:
import WAE.WAE_hyperparams as wae_hp
import subprocess

In [ ]:
import os
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, precision_score, recall_score
from scipy.stats import sem, t
import subprocess

# Caminho para os dados pré-processados
data_dir = "Data"
machine_types = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
n_folds = 10

# Função para rodar o pipeline de K-fold cross-validation
def run_kfold_pipeline(machine_type, machine_id):
    roc_aucs, f1s, accs, precs, recalls = [], [], [], [], []
    for fold in range(n_folds):
        # Folds para treino e teste
        test_fold = fold
        train_folds = [f for f in range(n_folds) if f != test_fold]

        # Carregar ciclos normais de treino (de todos os folds exceto o de teste)
        train_cycles = []
        for tr_fold in train_folds:
            train_path = os.path.join(data_dir, f"preprocessed_mimii_{machine_type}_{machine_id}_Fold_{tr_fold}", f"{machine_type}_{machine_id}_train_cycles_fold{tr_fold}.pkl")
            with open(train_path, "rb") as f:
                train_cycles += pickle.load(f)
        # Salvar temporariamente para o WAE-GAN
        train_tmp_path = f"tmp_{machine_type}_{machine_id}_train.pkl"
        with open(train_tmp_path, "wb") as f:
            pickle.dump(train_cycles, f)

        # Carregar ciclos de teste e labels do fold de teste
        test_path = os.path.join(data_dir, f"preprocessed_mimii_{machine_type}_{machine_id}_Fold_{test_fold}", f"{machine_type}_{machine_id}_test_cycles_fold{test_fold}.pkl")
        test_labels_path = os.path.join(data_dir, f"preprocessed_mimii_{machine_type}_{machine_id}_Fold_{test_fold}", f"{machine_type}_{machine_id}_test_labels_fold{test_fold}.pkl")
        with open(test_path, "rb") as f:
            test_cycles = pickle.load(f)
        with open(test_labels_path, "rb") as f:
            test_labels = pickle.load(f)
        test_tmp_path = f"tmp_{machine_type}_{machine_id}_test.pkl"
        with open(test_tmp_path, "wb") as f:
            pickle.dump(test_cycles, f)

        # Use subprocess para acompanhar os processos no terminal
        subprocess.run(f"python WAE/WAE_hyperparams.py all {machine_type} {machine_id} {fold}", shell=True)
        subprocess.run(f"python anomaly_detectionAdversarial.py --results_file results/{machine_type}_{machine_id}_Fold{fold}.pkl  --test_labels_path {test_labels_path}", shell=True)

        # Carregar resultados do fold
        results_file = f"results/{machine_type}_{machine_id}_Fold{fold}_metrics.pkl"
        if os.path.exists(results_file):
            with open(results_file, "rb") as f:
                metrics = pickle.load(f)
            roc_aucs.append(metrics["roc_auc"])
            f1s.append(metrics["f1"])
            accs.append(metrics["accuracy"])
            precs.append(metrics["precision"])
            recalls.append(metrics["recall"])

        # Limpar arquivos temporários
        os.remove(train_tmp_path)
        os.remove(test_tmp_path)

    # Calcular média, desvio padrão e intervalo de confiança para ROC-AUC
    roc_auc_mean = np.mean(roc_aucs)
    roc_auc_std = np.std(roc_aucs)
    conf_int = t.interval(0.95, len(roc_aucs)-1, loc=roc_auc_mean, scale=sem(roc_aucs))
    print(f"{machine_type}/{machine_id} - ROC-AUC: {roc_auc_mean:.4f} ± {roc_auc_std:.4f} (95% CI: {conf_int})")
    print(f"F1-score: {np.mean(f1s):.4f} ± {np.std(f1s):.4f}")
    print(f"Acurácia: {np.mean(accs):.4f} ± {np.std(accs):.4f}")
    print(f"Precisão: {np.mean(precs):.4f} ± {np.std(precs):.4f}")
    print(f"Recall: {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")

# Exemplo para rodar para todos os tipos/ids de máquina
for machine_type in ["valve"]:  
    for machine_id in ["id_00"]:  
        run_kfold_pipeline(machine_type, machine_id)

: 